In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install git+https://github.com/openai/whisper.git --use-deprecated=legacy-resolver
!sudo apt update && sudo apt install ffmpeg


In [ ]:
import glob
import os
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

audio_dir = "" # 音频文件夹
output_dir = ""    # 输出文件夹

# 查找最新的音频文件
latest_audio_file = max(glob.glob(f"{audio_dir}/*.m4a"), key=os.path.getctime)
print(latest_audio_file)

# 提取文件名（不包含扩展名）作为输出文件夹名
output_subdir = os.path.join(output_dir, os.path.splitext(os.path.basename(latest_audio_file))[0])
print(output_subdir)

# 执行whisper命令
!whisper "{latest_audio_file}" --model medium --language zh --output_dir "{output_subdir}"

# 查看帮助
# !whisper --h 

In [20]:
# 完成任务发送邮件通知
def send_email_notification(subject, body, to_email):
    from_email = "" # 发送邮件的邮箱
    from_password = "" # 发送邮件的邮箱的专用密码 16位授权码
    msg = MIMEMultipart()
    msg["From"] = from_email
    msg["To"] = to_email
    msg["Subject"] = subject

    # 添加邮件内容
    msg.attach(MIMEText(body, "plain"))

    # 使用Gmail的SMTP服务器发送邮件
    try:
        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.starttls()
        server.login(from_email, from_password)
        text = msg.as_string()
        server.sendmail(from_email, to_email, text)
        server.quit()
        print("邮件已发送!")
    except Exception as e:
        print(f"邮件发送失败: {e}")

# 调用函数发送通知邮件
# 获取简化后的路径
simplified_audio_file = "/".join(latest_audio_file.split("/")[-2:])  # 获取文件的最后两级路径
simplified_output_dir = "/".join(output_subdir.split("/")[-3:])  # 获取输出目录的最后三级路径

send_email_notification(
    subject= latest_audio_file.split("/")[-1] + " 会议音频字幕任务已完成",
    body=f"音频文件 {simplified_audio_file} 的转录已完成，结果保存在 /{simplified_output_dir}",
    to_email=""  # 接收邮件的邮箱
)


邮件已发送!
